In [2]:
import seaborn

import yfinance as yf
import pandas as pd
import openpyxl
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import statsmodels
import seaborn as sns
import scipy
from scipy import stats
from statistics import mean


import statsmodels.tsa.api as tsa
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, q_stat, adfuller
from scipy.stats import probplot, moment
import requests

import graphviz
import pydotplus

import eikon as ek  # the Eikon Python wrapper package
import configparser as cp

import sklearn
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
# Load libraries
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint


In [3]:
df = pd.read_csv("evaluation.csv")
df = df.dropna()
df

,Unnamed: 0,ExponentialMovingAverage,WeightedMovingAverage,StochasticSlow,MACDHisto,RSI,SmoothedMovingAverage,ATR,time,value,AO,WilliamR
33,33,22.272977,21.817292,23.841060,-0.918013,39.884703,41.656040,0.891948,2012-05-02,1.000000e+07,-1.970281,-78.807947
34,34,22.101210,21.629477,17.849824,-0.961326,34.859427,40.976378,0.897523,2012-05-03,1.000000e+07,-1.942963,-96.119403
35,35,21.901116,21.412308,9.237057,-1.027382,32.062684,40.085009,0.879129,2012-05-04,1.000000e+07,-2.078287,-97.361478
36,36,21.711031,21.203385,4.833056,-1.073011,31.707557,39.247264,0.888477,2012-05-07,1.000040e+07,-2.242728,-92.019950
37,37,21.532490,21.005077,5.914418,-1.099724,31.492901,38.471827,0.846800,2012-05-08,1.000059e+07,-2.445313,-92.875318
...,...,...,...,...,...,...,...,...,...,...,...,...
2807,2807,46.789227,47.166400,62.112587,0.661628,55.296724,56.222562,1.205445,2023-05-11,1.011187e+07,1.698687,-44.402277
2808,2808,46.867748,47.260862,59.898798,0.644233,56.331946,56.233501,1.156485,2023-05-12,1.011187e+07,1.711540,-40.607211
2809,2809,46.996383,47.397015,62.745098,0.681497,59.975287,56.607679,1.133879,2023-05-15,1.011187e+07,1.663628,-26.755218
2810,2810,47.065122,47.469477,64.516129,0.651074,55.532473,56.500159,1.115388,2023-05-16,1.011187e+07,1.580319,-39.089184


In [4]:
price_df = pd.read_csv("data.csv")
price_df




,Unnamed: 0,Datetime,Open,High,Low,Close,Volume
0,0,2012-03-15,23.00,23.500,22.700,23.40,13911300.0
1,1,2012-03-16,23.65,24.930,23.550,24.55,2063000.0
2,2,2012-03-19,24.50,25.000,24.060,24.85,974300.0
3,3,2012-03-20,24.71,25.200,24.250,24.80,444400.0
4,4,2012-03-21,24.65,25.180,24.130,24.25,782300.0
...,...,...,...,...,...,...,...
2808,2808,2023-05-12,47.69,47.980,47.460,47.81,525703.0
2809,2809,2023-05-15,48.00,48.620,47.780,48.54,569858.0
2810,2810,2023-05-16,48.33,48.405,47.665,47.89,748986.0
2811,2811,2023-05-17,48.12,48.840,47.830,48.63,666228.0


In [5]:
df["Closing price"] = price_df["Close"]

In [6]:
df.columns

Index(['Unnamed: 0', 'ExponentialMovingAverage', 'WeightedMovingAverage',
       'StochasticSlow', 'MACDHisto', 'RSI', 'SmoothedMovingAverage', 'ATR',
       'time', 'value', 'AO', 'WilliamR', 'Closing price'],
      dtype='object')

In [7]:
df["Short_window"] = df["Closing price"].rolling(3).mean
df["Short_window"] = df["Closing price"].rolling(3).mean

In [7]:
#df.drop(['Closing prrce'])
print(df["Closing price"])
#A function to compute a new pandas series for percentage changes of items inside a list
def pct_change(lst):
    changes = []
    for i in range(len(lst)):
        if i > 0:
            changes.append((lst[i] - lst[i-1])/lst[i-1])
        else:
            changes.append(0)

    df = pd.DataFrame(changes, index=None)
    return df

def turn_trend_into_bin(lst):
    lst = list(lst)
    bin = []
    for i in lst:
        if i > 0:
            bin.append(1)
        else:
            bin.append(-1)

    return pd.DataFrame(bin, index = None)

def range_interval(lst, high, low):
    lst = list(lst)
    ter = []
    for i in lst:
        if i >= high:
            ter.append(1)
        elif i > low:
            ter.append(0)
        else:
            ter.append(-1)

    return pd.DataFrame(ter, index = None)


#changes mean
short_window = 3
long_window = 4

df["Short_window"] = pct_change(list(df["Closing price"])).rolling(short_window).mean()
df["Long_window"] = pct_change(list(df["Closing price"])).rolling(long_window).mean()
# df["RSI_pct_change"] = pct_change(list(df["RSI"]))

df["RSI_range"] = range_interval(df["RSI"], 65, 38)
df["WMV_pct_change"] = pct_change(list(df['WeightedMovingAverage']))
df["WilliamR_range"] = range_interval(df["WilliamR"], -80, -20)

25      21.21
26      21.90
27      20.91
28      20.84
29      20.50
        ...  
2801    47.47
2802    46.08
2803    47.24
2804    48.10
2805    48.16
Name: Closing price, Length: 2781, dtype: float64


In [8]:
# df["Short_window"].plot()
# df["Long_window"].plot()

df["Short window bin"] = turn_trend_into_bin(df["Short_window"])
df["Long window bin"] = turn_trend_into_bin(df["Long_window"])
# df["RSI_bin"] = turn_trend_into_bin(df["RSI_pct_change"])
df["WMV_bin"] = turn_trend_into_bin(df["WMV_pct_change"])


df = df.fillna(method="ffill")

In [9]:
#List out combinations that might help
comb_1 = ['WeightedMovingAverage', 'RSI_range']
feature_cols = ['ExponentialMovingAverage', 'WeightedMovingAverage','StochasticSlow', 'MACDHisto', 'RSI', 'SmoothedMovingAverage', 'ATR']

using = ["RSI_range", "WMV_bin"]
X = df[using].values # Features
y = df["Short window bin"].values # Target variable

# df_2 = pd.read_csv("evaluation2.csv")
# X_test = df_2[feature_cols].dropna().values
# y_test = df_2["value"].dropna().values

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [11]:
#Random forest classifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.6822262118491921


In [21]:
import data_api
connection = data_api.API_connection()

df_raw = connection.backtester_data_usage("daily", "AXL", "NA")
print(df_raw)

df = pd.read_csv("evaluation2.csv")


df["Closing price"] = df_raw["Close"]

df["Short_window"] = pct_change(list(df["Closing price"])).rolling(short_window).mean()
df["Long_window"] = pct_change(list(df["Closing price"])).rolling(long_window).mean()
# df["RSI_pct_change"] = pct_change(list(df["RSI"]))

df["RSI_range"] = range_interval(df["RSI"], 65, 38)
df["WMV_pct_change"] = pct_change(list(df['WeightedMovingAverage']))

df["Short window bin"] = turn_trend_into_bin(df["Short_window"])
df["Long window bin"] = turn_trend_into_bin(df["Long_window"])
# df["RSI_bin"] = turn_trend_into_bin(df["RSI_pct_change"])
df["WMV_bin"] = turn_trend_into_bin(df["WMV_pct_change"])

df = df.fillna(method="ffill")

X_test = df[using].values # Features
# X_test = X_test[using]

TIME_SERIES_DAILY_ADJUSTED
https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=AXL&outputsize=full&apikey=FQ4QXSG40SMEWLCB
        Datetime   Open     High     Low  Close    Volume
0     1999-11-01  13.94  13.9400  13.500  13.63   11800.0
1     1999-11-02  13.75  13.8800  13.750  13.75   11000.0
2     1999-11-03  13.88  13.9400  13.750  13.94    8500.0
3     1999-11-04  13.88  13.9400  13.750  13.94  105400.0
4     1999-11-05  13.88  13.9400  13.810  13.94   13300.0
...          ...    ...      ...     ...    ...       ...
5918  2023-05-10   7.26   7.3000   7.020   7.11  829003.0
5919  2023-05-11   7.03   7.2350   7.005   7.17  723673.0
5920  2023-05-12   7.25   7.2600   7.000   7.11  710666.0
5921  2023-05-15   7.20   7.2275   7.100   7.15  519140.0
5922  2023-05-16   6.99   7.0700   6.850   6.87  747756.0

[5923 rows x 6 columns]


In [22]:

y_test = df["Short window bin"]

y_pred = rf.predict(X_test)
result_df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
result_df

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5103276353276354


In [28]:
pred = rf.predict(pd.DataFrame({"RSI_range":0, "WMV_bin":-1}, index=[0]))
pred

C:\Users\tszki\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


array([-1.])

In [ ]:
# Export the first three decision trees from the forest

for i in range(3):
    tree = rf.estimators_[i]
    dot_data = export_graphviz(tree,
                               feature_names=X_train.columns,
                               filled=True,
                               max_depth=2,
                               impurity=False,
                               proportion=True)
    graph = graphviz.Source(dot_data)
    display(graph)

In [ ]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X, y)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [ ]:
print("Accuracy:",sklearn.metrics.accuracy_score(y_test, y_pred))

In [ ]:
pip install graphviz

In [ ]:
from sklearn.tree import export_graphviz
from six import StringIO
from IPython.display import Image
import pydotplus

dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,
                filled=True, rounded=True,
                special_characters=True,feature_names = using,class_names=None)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_png('diabetes.png')
Image(graph.create_png())

In [ ]:
from sklearn import preprocessing as pp

sc = pp.StandardScaler()
normed_train_data = pd.DataFrame(sc.fit_transform(X_train), columns = ["RSI_TER", "WMV_bin"])
normed_test_data = pd.DataFrame(sc.fit_transform(X_test), columns = ["RSI_TER", "WMV_bin"])

In [ ]:
testing_set = df[['ExponentialMovingAverage', 'WeightedMovingAverage','StochasticSlow', 'MACDHisto', 'RSI', 'SmoothedMovingAverage', 'ATR']]

In [ ]:
import seaborn

seaborn.pairplot(testing_set)